In [1]:
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from tqdm import tqdm
from dotenv import dotenv_values

In [2]:
class VAE(nn.Module):
    def __init__(self, latent_dim=72, input_size=1326, dropout=0.2):
        super(VAE, self).__init__()
        self.dropout = dropout
        
        # Encoder Layers
        self.enc1 = nn.Linear(input_size,1024)
        self.enc2 = nn.Linear(1024,512)
        self.enc3 = nn.Linear(512,256)
        self.enc4 = nn.Linear(256,128)
        
        self.fc1 = nn.Linear(128, latent_dim)
        self.fc_mu = nn.Linear(latent_dim, latent_dim)
        self.fc_log_var = nn.Linear(latent_dim, latent_dim)
        self.fc2 = nn.Linear(latent_dim, 128)
        
        # Decoder Layers
        self.dec1 = nn.Linear(128,256)
        self.dec2 = nn.Linear(256,512)
        self.dec3 = nn.Linear(512,1024)
        self.dec4 = nn.Linear(1024,input_size)
    
    def reparameterize(self, mu, log_var):
        std = torch.exp(0.5*log_var)
        eps = torch.randn_like(std)
        return mu + (eps * std)
    
    def forward(self, x):
        x = F.dropout(F.relu(self.enc1(x)), p=self.dropout)
        x = F.dropout(F.relu(self.enc2(x)), p=self.dropout)
        x = F.dropout(F.relu(self.enc3(x)), p=self.dropout)
        x = F.dropout(F.relu(self.enc4(x)), p=self.dropout)
        hidden = self.fc1(x)
        
        mu = self.fc_mu(hidden)
        log_var = self.fc_log_var(hidden)
        
        z = self.reparameterize(mu, log_var)
        z = self.fc2(z)
        
        x = F.dropout(F.relu(self.dec1(z)), p=self.dropout)
        x = F.dropout(F.relu(self.dec2(x)), p=self.dropout)
        x = F.dropout(F.relu(self.dec3(x)), p=self.dropout)
        reconstruction = self.dec4(x)
        
        return reconstruction, mu, log_var

In [3]:
def final_loss(bce_loss, mu, logvar):
    BCE = bce_loss
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return BCE + KLD

In [4]:
config = dotenv_values('../.env')
FEATURES = [str(c) for c in range(1326)]
device = torch.device('cuda' if torch.cuda.is_available else 'cpu')
# device = torch.device('cpu')
model = VAE().to(device)

lr = 1e-3
batch_size = 128
optimizer = optim.Adam(model.parameters(),lr=lr)
criterion = nn.MSELoss()
epochs = 5

In [6]:
# Train
model.train()

for epoch in range(epochs):
    print(f"Epoch {epoch}:")
    running_loss = 0.0
    counter = 0
    for chunk in tqdm(pd.read_csv(config["ENGINEERED_DATA"] + "imputed_train.csv",
                             chunksize=batch_size, usecols=FEATURES)):
        counter += 1
        data = torch.Tensor(chunk.values)
        data = data.to(device)
        optimizer.zero_grad()
        reconstruction, mu, logvar = model(data)
        loss = F.binary_cross_entropy(reconstruction, data) + \
               F.kl_div(reconstruction, data, reduction='batchmean')
        loss.backward()
        running_loss += loss.item()
        print(running_loss)
        optimizer.step()
    print(f"Loss: {running_loss/counter}")

Epoch 0:


0it [00:00, ?it/s]


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [ ]:
# Validate
